In [0]:
!pip install deeppavlov
!pip install tensorflow==1.15.0
!apt-get --yes install git
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
import pandas as pd
import json

In [0]:
from deeppavlov import configs, build_model, train_model

In [0]:
with configs.ner.ner_ontonotes_bert_mult.open(encoding='utf8') as file:
    ner_config = json.load(file)

In [0]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [0]:
ner_config['dataset_reader']['data_path'] = './'  # directory with train.txt, valid.txt and test.txt files
ner_config['metadata']['variables']['NER_PATH'] = './'
ner_config['metadata']['download'] = [ner_config['metadata']['download'][-1]]  # do not download the pretrained ontonotes model

In [0]:
ner_model = train_model(ner_config, download=True)

In [16]:
ner_model(['Playstation 4', 'Xbox 360 продам', 'Продам ПС 3'])

[[['Playstation', '4'], ['Xbox', '360', 'продам'], ['Продам', 'ПС', '3']],
 [['B-PRS', 'I-PRS'], ['B-PRS', 'I-PRS', 'O'], ['O', 'O', 'O']]]

In [0]:
marked = []

for text in data.text.values[:1000]:
    # BERT имеет лимит на длину текста в 512 слов, возьмем даже еще меньше
    if len(text.split()) > 100:
        continue
    pred = ner_model([text])
    sent, tags = pred[0][0], pred[1][0]
    
    # достанем только тексты с сущностями
    if len(set(tags[0])) > 1:
        marked.append(list(zip(sent,tags)))